In [ ]:
!pip install redis

# TODO: publish/subscribe function within redis

In [ ]:
from flask import Flask, request, jsonify, redirect, render_template
import redis
import pandas as pd
import datetime as dt

In [ ]:
# Connect to redis
redis_db = redis.Redis(host='localhost', port=6379)

In [ ]:
## Create Flask API
api = Flask(__name__)

#### CREATE STOCKMENTION

Key = ticker:date \
Value(Hash) = (price, mention) \

In [ ]:
# Delete all in redis
#redis_db.flushall()
#len(redis_db.keys())

In [ ]:
@api.route("/ticker/<string:ticker>/<string:date>/<int:price>/<int:mentions>", methods=["POST"])
def insert_ticker_values(ticker, date, price, mentions):
    expire_date = date.datetime.datetime.strptime(start_date, "%m/%d/%y") + datetime.timedelta(days=7)
    time_to_expire_s = str(dt.datetime.strptime(expire_date, "%Y-%m-%d").timestamp())
    
    values = {"price":price, "mentions":mentions}
    try:
        transaction = redis_db.pipeline(transaction=True)
        #transaction.sadd("ticker", date_utc)
        transaction.hmset(ticker +':'+ date, values, ex=time_to_expire_s)        
        transaction.execute()
        return(f"values for "+ticker+" were inserted")
    except Exception as e:
        print(e)

## GET STOCK OVERVIEW

In [ ]:
@api.route("/weekly/<string:ticker>", methods=["GET"])
def get_weekly(ticker): 
    #utc_from = dt.datetime.strptime(date_from, "%Y-%m-%d").timestamp()
    #utc_to = dt.datetime.strptime(date_to, "%Y-%m-%d").timestamp()
   # if redis_db.exists(ticker):
    weekly_overview = pd.DataFrame()
    for key in redis_db.keys("GME:*"):
        key = key.decode('utf-8')
        date = key[4:]
        values = redis_db.hmget(key, 'price', 'mentions')
        
        dayly_overview = {
            'Date': date,
            'Price': values[0].decode('utf-8'),
            'Mentions': values[1].decode('utf-8')
        }
        weekly_overview = weekly_overview.append(dayly_overview, ignore_index=True)
        weekly_overview.sort_values('Date')
        
    return weekly_overview.to_json(orient='records')

In [ ]:
insert_ticker_values('GME', '2021-12-11', '78', '16')
insert_ticker_values('GME', '2021-12-12', '75', '23')
insert_ticker_values('GME', '2021-12-13', '74', '15')
insert_ticker_values('GME', '2021-12-14', '73', '24')
insert_ticker_values('GME', '2021-12-04', '73', '24')

In [ ]:
#df = get_week('GME')

#df.sort_values('Date')

In [ ]:
get_weekly('GME')


In [ ]:
values = redis_db.hmget('GME', 'price', 'mentions')
values[0]

In [ ]:
weekmap = redis_db.keys(pattern='ticker:*')
len(weekmap)

In [ ]:
redis_db.hset("date", 3, 3)
redis_db.hset("date", 5, 5)

In [ ]:

mymap = redis_db.keys(pattern='ticker.*')
len(mymap)

In [ ]:
if __name__ == "__main__":
    api.run(port=5051)